In [12]:
import json
import torch
import torch.nn as nn
from torchvision import transforms, models
from PIL import Image

In [13]:
# Load class names
with open('class_names.json', 'r') as f:
    class_names = json.load(f)

In [14]:
# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [15]:
# Define the model architecture
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
model.fc = nn.Linear(model.fc.in_features, len(class_names))  # Adjust final layer
model = model.to(device)

In [16]:
# Load the saved model state dictionary
model_path = 'plant_disease_model.pth'
model.load_state_dict(torch.load(model_path))
model.eval()  # Set the model to evaluation mode
print(f"Model loaded from {model_path}")

Model loaded from plant_disease_model.pth


In [17]:
# Image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])


In [7]:
# Function to predict a single image
def predict_image(image_path, model, transform):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)
    
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
    
    return predicted.item()

In [25]:
# Example usage
image_path = 'Potato___healthy_batch473_img1.jpg'
predicted_class = predict_image(image_path, model, transform, class_names)
print(f"Predicted class: {predicted_class}")

Predicted class: Potato___healthy
